In [1]:
import numpy as np
import pandas as pd
from noise import *
import random
import os 
from sklearn.model_selection import train_test_split  # Import train_test_split

In [2]:
# x values
def generate_x_values(start_x, end_x, n_datapoints, n_samples):
    # create n x values
    xs = np.linspace(start_x, end_x, n_datapoints)
    xs = np.tile(xs, (n_samples, 1))
    return xs

In [3]:
# True functions
def sin(xs):
    return np.sin(xs)

In [4]:
# Noise functions
def pink(xs, amplitude):
    y_added_noise = np.zeros(xs.shape)
    for sample in range(xs.shape[0]):
        y_added_noise[sample, :] = amplitude*pink_noise(xs[sample, :])
    return y_added_noise

def uniform(xs, lower_bound, upper_bound):
    return np.random.uniform(lower_bound, upper_bound)

def gaussian(xs, std):
    return np.random.normal(0, std, xs.shape)

In [5]:
# SNR
def calculate_snr(y_true, y_noise): ### write own code later
    # Calculate signal power
    signal_power = np.mean(y_true ** 2)
    # Calculate noise power
    noise_power = np.mean((y_noise - y_true) ** 2)
    # Calculate (linear) SNR
    snr = signal_power / noise_power
    # Convert to dB
    snr_db = 10 * np.log10(snr)
    return snr, snr_db

In [ ]:
# Dataset parameters
n_datapoints = 20  # number of (total) datapoints generated
n_samples = 100
start_x, end_x = -10, 10  # x values to be used

# Train/validation/test split parameters
train_size = 0.7
validation_size = 0.2
test_size = 0.1

# Function parameters
func = sin  # Available: x, x_sq, sin

# Noise parameters
noise = pink  #  #TODO Available: pink, uniform, gaussian

amplitude = None  
upper_bound, lower_bound = None, None
std = None

if noise == pink:
    amplitude = 0.05 # We want values: amplitude in (0.01, 20), to get snr_db in approx. (-30, 40) for sin(x) 
elif noise == uniform:
    upper_bound, lower_bound = -0.01, 0.01  # We want values: upper_bound, lower_bound in (-0.01, 0.01) to (-20, 20), to ge snr_db in approx. (-30, 40) for sin(x)
elif noise == gaussian:
    std = 1 #Mellan [0.01, 100]

In [7]:
# Dataset generation
xs = generate_x_values(start_x, end_x, n_datapoints, n_samples)  # x values
#print(xs.shape)
y_true = func(xs)

if noise == pink:
    y_added_noise = noise(xs, amplitude)
elif noise == uniform:
    y_added_noise = noise(xs, lower_bound, upper_bound)
elif noise == gaussian:
    y_added_noise = noise(xs, std)

y_noise = y_true + y_added_noise

# Calculate snr, snr_db for the entire dataset
snr, snr_db = calculate_snr(y_true, y_noise)